<a href="https://colab.research.google.com/github/s-kamireddy/AI_HW_6/blob/main/AI_HW_6_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install datasets

In [ ]:
!pip install rouge-score

In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
import torch
from torchvision import datasets
import datasets

In [ ]:

import pandas as pd
import torch
from transformers import GPT2Tokenizer
import warnings
warnings.filterwarnings('ignore')

Data exploration and preprocessing

In [ ]:
dataset = datasets.load_dataset("cnn_dailymail", "3.0.0")

train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
test_dataset = dataset["test"].shuffle(seed=42).select(range(500))
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(500))

#print
print(train_dataset["highlights"][0])
print(test_dataset)

John and .
Audrey Cook were discovered alongside their daughter, Maureen .
They were found at Tremarle Home Park in Cornwall .
Investigators say the three died of carbon monoxide .
poisoning .
Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 500
})


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizerFast, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.optim import Adam
import os



In [ ]:
maxInput = 512
maxTarget = 128
batchSize = 3
modelCheckpoint = "facebook/bart-large-cnn"

In [ ]:
model = BartForConditionalGeneration.from_pretrained(modelCheckpoint)
tokenizer = BartTokenizerFast.from_pretrained(modelCheckpoint)

In [ ]:
def preprocess_function(examples):
    inputs = [text for text in examples["article"]]
    model_inputs = tokenizer(inputs, max_length= 512,padding = 'max_length', truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128,padding = 'max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_eval = eval_dataset.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
print(tokenized_train[0])
print(tokenized_test)

{'input_ids': tensor([    0,  2765,   479,  3173,  7291,   479, 29731,  7976, 14849,  1691,
           35,   479, 13470,    35,  3933, 12936,     6,   132,   494,  1014,
          479,  1721,   479,   121, 44964,    35,   479, 12112,    35,  3570,
        12936,     6,   132,   494,  1014,   479,  2873,   453,     9,     5,
          276,   284,    54,   962,    11,    10, 25156, 15987,    31,  4363,
         6154, 24260, 15000,    74,    33,    57, 15003,   128, 34451,   728,
         3934,  3725,    26,   452,     4,    20,  3738,     9,  2997,   891,
          610,     8, 25478,  4350,    58,  2967,  2863,    49,  1354,     6,
         3066, 26743,     6,    23,     5,  1830,   184,    51,  1373,    15,
        31103,   271,   459,  2193,   861,    11,  4536, 17080,     6,  3072,
        21690,     4,    20, 27874, 10092,    33,   122,  1357,    88,     5,
         3257,    94,   378,     6,    19,  3725,   584,     5,   130,   962,
          552,    19,     5,   284,    18,  4716, 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

def training(dataset, model, tokenizer, batch_size=8, epochs=10, learning_rate=1e-3, accumulation_steps=4):
    import torch
    from torch.utils.data import DataLoader
    from torch.optim import Adam
    from transformers import get_linear_schedule_with_warmup

    torch.cuda.empty_cache()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    batch_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = Adam(model.parameters(), lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps= (len(batch_loader) // accumulation_steps + int(len(batch_loader) % accumulation_steps > 0)) * epochs)

    for epoch in range(epochs):
        print(f"Epoch {epoch}/{epochs}")
        running_loss = 0.0
        optimizer.zero_grad()

        for idx, batch in enumerate(batch_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / accumulation_steps
            loss.backward()
            running_loss += loss.item()

            if (idx + 1) % accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()  # Reset gradients for the next accumulation cycle
                print(f"Batch {idx + 1}/{len(batch_loader)} - Loss: {running_loss}")
                running_loss = 0.0

            del input_ids
            del attention_mask
            del labels
            del outputs
            del loss
            torch.cuda.empty_cache()

        #last few batches
        if len(batch_loader) % accumulation_steps != 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            print(f"End of epoch {epoch + 1} - Loss: {running_loss}")

    return model



In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
model = training(tokenized_train, model, tokenizer, batch_size=2, epochs=1, learning_rate=1e-3)

Epoch 0/1
Batch 4/500 - Loss: 6.380910038948059
Batch 8/500 - Loss: 15.171371936798096
Batch 12/500 - Loss: 11.559797286987305
Batch 16/500 - Loss: 10.052970170974731
Batch 20/500 - Loss: 9.458039999008179
Batch 24/500 - Loss: 8.634066104888916
Batch 28/500 - Loss: 7.938820123672485
Batch 32/500 - Loss: 6.535948276519775
Batch 36/500 - Loss: 6.437809467315674
Batch 40/500 - Loss: 6.666896224021912
Batch 44/500 - Loss: 6.055567145347595
Batch 48/500 - Loss: 5.952527761459351
Batch 52/500 - Loss: 6.463042259216309
Batch 56/500 - Loss: 5.091938853263855
Batch 60/500 - Loss: 5.467901587486267
Batch 64/500 - Loss: 5.833650469779968
Batch 68/500 - Loss: 4.998159170150757
Batch 72/500 - Loss: 5.219856858253479
Batch 76/500 - Loss: 5.343723297119141
Batch 80/500 - Loss: 4.9865193367004395
Batch 84/500 - Loss: 5.290077328681946
Batch 88/500 - Loss: 5.680011808872223
Batch 92/500 - Loss: 4.704928994178772
Batch 96/500 - Loss: 5.733253479003906
Batch 100/500 - Loss: 4.874337434768677
Batch 104/50

KeyboardInterrupt: 

In [ ]:
# Saving the model state after training is complete
torch.save(
        model.state_dict(),
        os.path.join(".", "model-final.pt"),
)


Evaluating Model

In [ ]:
#load model
model.load_state_dict(torch.load(os.path.join(".", "model-final.pt")))

<All keys matched successfully>

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
torch.cuda.empty_cache()
# Instantiate the Rouge class
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

model.to('cuda')
model.eval()

all_predictions = []
all_labels = []

test_loader = DataLoader(tokenized_test, batch_size=2, shuffle=True)

for idx, batch in enumerate(test_loader):
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)

    predictions = outputs
    all_predictions.extend(predictions)
    all_labels.extend(labels)

    if idx % 10 == 0:
        print(f"Batch {idx + 1}/{len(test_loader)}")

    del input_ids
    del attention_mask
    del labels
    del outputs
    torch.cuda.empty_cache()





Batch 1/250
Batch 11/250
Batch 21/250
Batch 31/250
Batch 41/250
Batch 51/250
Batch 61/250
Batch 71/250
Batch 81/250
Batch 91/250
Batch 101/250
Batch 111/250
Batch 121/250
Batch 131/250
Batch 141/250
Batch 151/250
Batch 161/250
Batch 171/250
Batch 181/250
Batch 191/250
Batch 201/250
Batch 211/250
Batch 221/250
Batch 231/250
Batch 241/250


In [ ]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [ ]:
%pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.7 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load('sacrebleu')
scores = {'rouge1':[], 'rouge2':[], 'rougeL':[], 'bleu':[]}

for prediction, label in zip(all_predictions, all_labels):
    pred_text = tokenizer.decode(prediction, skip_special_tokens=True)
    label_text = tokenizer.decode(label, skip_special_tokens=True)

    score = scorer.score(label_text, pred_text)

    #bleu_score = bleu.compute(predictions=[pred_text], references= [[label_text]])

    scores['rouge1'].append(score['rouge1'].fmeasure)
    scores['rouge2'].append(score['rouge2'].fmeasure)
    scores['rougeL'].append(score['rougeL'].fmeasure)
    scores['bleu'].append('bleu_score')





print('Rouge1:', sum(scores['rouge1']) / len(scores['rouge1']))
print('Rouge2:', sum(scores['rouge2']) / len(scores['rouge2']))
print('RougeL:', sum(scores['rougeL']) / len(scores['rougeL']))
#print('Bleu:',sum(scores['bleu']) / len(scores['bleu']))

Rouge1: 0.17937228376500686
Rouge2: 0.004593414297842033
RougeL: 0.11295938656114883


In [ ]:
for prediction, label in zip(all_predictions, all_labels):
    pred_text = tokenizer.decode(prediction, skip_special_tokens=True)
    label_text = tokenizer.decode(label, skip_special_tokens=True)
    bleu_score = bleu.compute(predictions=[pred_text], references= [[label_text]])

    scores['bleu'].append(bleu_score['score'])

print('Bleu:',sum(scores['bleu']) / len(scores['bleu']))



KeyboardInterrupt: 

Discussions on hyperparameters

Performance was not very accurate. Given the oppourtunity there is much that could be done to fine tune the hyperparameters for better performance, such as increasing the number of training epochs, increasing the embedding sizes, and decreasing the learning rate